In [ ]:
# torch with cuda
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from ultralytics import YOLO
import torch
import os

# Load a model
# model = YOLO("best.onnx")
model = YOLO("yolo11n.pt")

path = os.path.join(os.path.abspath("."),"License Plate Recognition.v6i.yolov11.zip")

# Train the model
train_results = model.train(
    data=path,  # path to dataset YAML
    epochs=100,  # number of training epochs
    # workers=100,
    # patience=20,
    imgsz=640,  # training image size
    device="cuda" if torch.cuda.is_available() else "cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
)

# Evaluate model performance on the validation set
metrics = model.val()

# Export the model to ONNX format
path = model.export(format="onnx")  # return path to exported model

In [ ]:
import cv2
from matplotlib import pyplot as plt

def run(img):
    results = model(img)
    results[0].show()
    print(results[0])

# Perform object detection on an image
img = cv2.imread("example.jpg")
run(img)
# img = cv2.flip(img, 0)
# run(img)
# img = cv2.flip(img, 1)
# run(img)
# img = cv2.flip(img, 0)
# run(img)

In [ ]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO

model_path = os.path.join(os.path.abspath("."),"models","first10ktrain","weights","best.onnx")
model = YOLO(model_path)

def add_noise(bboxes, mean=0, std_dev=5):
    noise = np.random.normal(mean, std_dev, bboxes.shape)
    noisy_bboxes = bboxes + noise
    return noisy_bboxes

def extract_bboxes(results):
    #bounding boxes can be extracted like that:
    bboxes = results[0].boxes.xyxy.cpu().numpy()

    #trying to add noise and add it on the picture (doesn't work yet)
    noisy_bboxes = add_noise(bboxes)
    results[0].boxes.xyxy = torch.tensor(noisy_bboxes, device=results[0].boxes.xyxy.device)

    return results

def run(img):
    results = model(img)

    #results = extract_bboxes(results)
    results[0].show()
    print(results[0])

# Perform object detection on an image
img = cv2.imread("example.jpg")
run(img)